In [1]:
from nga.utils import load_config, plane_eval_ray_bundle, save_as_image
from nerfstudio.utils.eval_utils import eval_load_checkpoint
from nerfstudio.cameras.rays import RayBundle
from nerfstudio.utils import colormaps

from PIL import Image
from pathlib import Path
import yaml
import torch
import json

In [6]:
# config_path = "outputs/pattern_plane1_720x480/tensorf/2023-09-01_085608/config.yml"
config_path = "outputs/pattern_plane1_720x480/instant-ngp/2023-08-31_194733/config.yml"
# config_path = "outputs/pattern_plane1_720x480/nerfacto/2023-09-01_092209/config.yml"
# config_path = "outputs/pattern_plane1_720x480/nerfacto/2023-09-01_102747/config.yml"


config = load_config(config_path)
config.load_dir = config.get_checkpoint_dir()

# setup pipeline (which includes the DataManager)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pipeline = config.pipeline.setup(device=device, test_mode="val")
pipeline.eval()

checkpoint_path, step = eval_load_checkpoint(config, pipeline)
results_path = config.get_base_dir() / "results.json"
render_output_path = config.get_base_dir() / "renders"
render_output_path.mkdir(parents=True, exist_ok=True)

# load data for coverting between original and normalized coodinate space
dataparser_transforms_path = config.get_base_dir() / "dataparser_transforms.json"
dataparser_transforms_data = json.load(open(dataparser_transforms_path))
dataparser_scale = dataparser_transforms_data["scale"]
transform = torch.tensor(dataparser_transforms_data["transform"])

near_z = 0.15
camera_ray_bundle = plane_eval_ray_bundle(dataparser_transforms_data, near_z).to(device)
outputs = pipeline.model.get_outputs_for_camera_ray_bundle(camera_ray_bundle)

rgb = outputs["rgb"]
acc = colormaps.apply_colormap(outputs["accumulation"])
depth = colormaps.apply_depth_colormap(
    outputs["depth"],
    accumulation=outputs["accumulation"],
)

z = near_z - (outputs["depth"] / dataparser_scale)
z_vis = colormaps.apply_depth_colormap(
    torch.clamp(z, -0.3, 0.3),
    accumulation=outputs["accumulation"],
)

save_as_image(rgb, render_output_path / "rgb.jpg")
save_as_image(acc, render_output_path / "acc.jpg")
save_as_image(depth, render_output_path / "depth.jpg")
save_as_image(z_vis, render_output_path / "z.jpg")
torch.save(z_vis, render_output_path / "z.pt")

# Get the output and define the names to save to
benchmark_info = {
    "experiment_name": config.experiment_name,
    "method_name": config.method_name,
    "checkpoint": str(checkpoint_path),
    "results": {
        "max_z": float(torch.max(z)),
        "min_z": float(torch.min(z)),
        "std_z": float(torch.std(z)),
        "mean_z": float(torch.mean(z)),
    },
}
# Save output to output file
results_path.write_text(json.dumps(benchmark_info, indent=2), "utf8")

[13:14:34] Auto image downscale factor of 1                                                 ]8;id=980137;file:///home/ccl/.conda/envs/nerfstudio/lib/python3.8/site-packages/nerfstudio/data/dataparsers/nerfstudio_dataparser.py\nerfstudio_dataparser.py]8;;\:]8;id=728513;file:///home/ccl/.conda/envs/nerfstudio/lib/python3.8/site-packages/nerfstudio/data/dataparsers/nerfstudio_dataparser.py#349\349]8;;\

            Dataset is overriding train_indices to [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, ]8;id=328755;file:///home/ccl/.conda/envs/nerfstudio/lib/python3.8/site-packages/nerfstudio/data/dataparsers/nerfstudio_dataparser.py\nerfstudio_dataparser.py]8;;\:]8;id=5440;file:///home/ccl/.conda/envs/nerfstudio/lib/python3.8/site-packages/nerfstudio/data/dataparsers/nerfstudio_dataparser.py#200\200]8;;\
           20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39,                              
           40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59,                              
           60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79,                              
           80, 81, 82, 83, 84, 85, 86, 87, 88, 89]                                                                      

            Dataset is overriding val_indices to [90, 91, 92, 93, 94, 95, 96, 97, 98, 99]   ]8;id=675172;file:///home/ccl/.conda/envs/nerfstudio/lib/python3.8/site-packages/nerfstudio/data/dataparsers/nerfstudio_dataparser.py\nerfstudio_dataparser.py]8;;\:]8;id=17568;file:///home/ccl/.conda/envs/nerfstudio/lib/python3.8/site-packages/nerfstudio/data/dataparsers/nerfstudio_dataparser.py#200\200]8;;\

Setting up training dataset...

Caching all 80 images.

...

Caching all 10 images.

Output()

Loading latest checkpoint from load_dir

✅ Done loading checkpoint from 
outputs/pattern_plane1_720x480/instant-ngp/2023-08-31_194733/nerfstudio_models/step-000029999.ckpt

359